In [1]:
include("deep-learning.jl")

update_params! (generic function with 2 methods)

# Mnist dataset

In [2]:
train_x, train_y = MNIST.traindata()
test_x,  test_y  = MNIST.testdata()
labels=[0,1,2,3,4,5,6,7,8,9]

10-element Array{Int64,1}:
 0
 1
 2
 3
 4
 5
 6
 7
 8
 9

In [3]:
train_y_one_hot=[onehot(label,labels) for label in train_y]

60000-element Array{Array{Int64,1},1}:
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
 ⋮
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [4]:
test_y_one_hot=[onehot(label,labels) for label in test_y]

10000-element Array{Array{Int64,1},1}:
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 ⋮
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [5]:
function train!(model,no_of_epoch=2,batch_size=256)
batch_size=256
data_size=size(train_x)[3]
for i in 1:no_of_epoch
    @printf("epoch %f \n", i)
    for j in 1:convert(Int,floor(data_size/batch_size))
        batch_indexes=rand(1:data_size,batch_size)
        x=unsqueeze(convert(Array{Float32},train_x[:,:,batch_indexes]),3)
        y=train_y_one_hot[batch_indexes]
        params=get_params(model)
        grads=Zygote.gradient(() -> (cross_entropy_loss_with_predict_batch(y,model,x)),params)
        update_params!(model,grads,0.001)
    end
    
    #loss
    total_loss=0.0
    for k in 1:size(train_x)[3]
        total_loss+=cross_entropy_loss_with_predict(train_y_one_hot[k],model,unsqueeze(convert(Array{Float64},train_x[:,:,k]),3))
    end
    
    @printf("total_loss %f \n", total_loss)
        
end

end

train! (generic function with 3 methods)

In [6]:
function accuracy(X,Y,model)
   
    valid = 0
    length = size(X)[3]
    for k in 1:length
        yPred=model(unsqueeze(convert(Array{Float32},X[:,:,k]),3))
        y=Y[k]
        if findmax(y)[2] == findmax(yPred)[2][2]
            valid=valid+1
        end

    end
    return valid/length
end

accuracy (generic function with 1 method)

In [7]:
model=Chain(Reshape(784),Dense(784,50),Relu(),Dense(50,10),SoftMax(2))

Chain((Reshape(784), Dense(Float32[1.7902685 0.43065593 … 0.8922561 1.5571667; -0.7758272 1.3565067 … 2.409027 0.20205589; … ; -0.27105176 0.39913464 … -1.5021553 1.2085733; 2.237659 -0.7938624 … -1.3952717 0.5969479], [0.0 0.0 … 0.0 0.0]), Relu(), Dense(Float32[0.9445156 0.74855506 … -0.034810208 -1.7600251; -0.5722169 -1.7961079 … 1.2898349 0.65176797; … ; 0.28019217 -0.4915944 … -0.98212194 -0.49810505; -0.7463173 1.4856942 … -1.592523 1.0631955], [0.0 0.0 … 0.0 0.0]), SoftMax(2)))

In [ ]:
train!(model,20)

epoch 1.000000 
total_loss 1385793.721100 
epoch 2.000000 
total_loss 1383484.606328 
epoch 3.000000 
total_loss 1382665.295761 
epoch 4.000000 
total_loss 1382319.020490 
epoch 5.000000 
total_loss 1382085.911473 
epoch 6.000000 
total_loss 1381945.174475 
epoch 7.000000 
total_loss 

In [ ]:
accuracy(train_x,train_y_one_hot,model)

# Debugging 

In [ ]:
m = Chain(Dense(784,50),Relu(),Dense(50,10),SoftMax(2))


In [ ]:
x = rand(Float32,1,784);

In [ ]:
x

In [ ]:
y=[1.0,0,0,0,0,0,0,0,0,0]

In [ ]:
(m(x))

In [ ]:
params=get_params(m)
grads=Zygote.gradient(() -> cross_entropy_loss_with_predict(y,m,x), params)

In [ ]:
params

In [ ]:
sum(grads[m.layers[3].weights])